In [6]:
import random
import time
import functools

### Decorators

In [22]:
def foo():
    time.sleep(random.randint(1, 2))

print('BEFORE')
foo()
print('AFTER')

BEFORE
AFTER


In [23]:
def foo2():
    start = time.time()

    time.sleep(random.randrange(0, 1))

    end = time.time()
    print(f'Elapsed {end - start}ms')

foo2()

Elapsed 2.002716064453125e-05ms


In [24]:
def foo3():
    start = time.time()

    lst = [i for i in range(10**6)]

    print(f'Elapsed {time.time() - start}ms')

foo3()

Elapsed 0.035893917083740234ms


In [25]:
def profile(f):
    start = time.time()

    f()

    print(f'Elapsed {time.time() - start}ms')

profile(foo)

Elapsed 2.001828193664551ms


In [31]:
type(foo())

NoneType

In [1]:
def say_hello():
    def internal():
        print('Hello')
    return internal

f = say_hello()
print(type(f))
f()

<class 'function'>
Hello


In [34]:
def say_hello():
    def internal(msg):
        print('Hello', msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello()
f1('Bill')

Hello John
Hello Bill


In [8]:
def say_hello(greeting='Hello'):
    def internal(msg):
        print(greeting, msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello('Hi')
f1('Bill')

Hello John
Hi Bill


In [9]:
def say_hello(greeting='Hello'):
    def internal(*args):
        print(greeting, *args)
    return internal

f1 = say_hello()
f1('John', 42, [1, 2, 3], True)

Hello John 42 [1, 2, 3] True


In [46]:
def profile(f):
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [37]:
# foo()
foo=profile(foo)
foo()

Elapsed time for function foo: 1.001065731048584ms
Elapsed time for function deco: 1.0011940002441406ms


In [48]:
@profile  # -> foo5 = profile(foo5)
def foo5():
    time.sleep(random.randint(1, 2))
    return 42

# foo5 = profile(foo5)
print(foo5())

42


### We need to go deeper

In [13]:
@profile
def foo6():
    time.sleep(random.randint(1, 2))
    return 42

print(foo6())

Elapsed time for function foo6: 2.0023670196533203ms
None


In [14]:
def profile(f):
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [51]:
@profile
def foo7():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo7())
help(foo7)

Elapsed time for function foo7: 2.001713275909424ms
42
Help on function deco in module __main__:

deco(*args)



In [52]:
def profile(f):
    @functools.wraps(f)
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [54]:
@profile
def foo8():
    """Help for foo8"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo8())
help(foo8)

Elapsed time for function foo8: 1.0017116069793701ms
42
Help on function foo8 in module __main__:

foo8()
    Help for foo8



In [56]:
@profile('Time spent') # -> profile_ = profile('Time spent')
                       # -> foo5 = profile_(foo5)

def foo8():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo8())
help(foo8)

Time spent foo8: 2.0019683837890625ms
42
Help on function foo8 in module __main__:

foo8()
    Help for foo7



In [55]:
def profile(msg):
    def profile_(f):
        @functools.wraps(f)
        def deco(*args):
            start = time.time()
            result = f(*args)
            print(msg, f'{f.__name__}: {time.time() - start}ms')
            return result
        return deco
    return profile_

In [84]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args):
            start = time.time()
            deco._num_call += 1
            result = f(*args)
            deco._num_call -= 1
            
            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}ms')
            return result
        
        deco._num_call = 0
        return deco
    
    return internal

In [57]:
def repeate(n=2):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            for _ in range(n):
                f(*args, **kwargs)
        return repeater
    return internal

@repeate(3)
def my_print(*args):
    print(*args)

my_print('Hello')


Hello
Hello
Hello


In [61]:
@profile()
@repeate()
def foo9():
    """Help for foo9"""
    time.sleep(random.randint(1, 2))
    return 42

foo9()

Elapsed time for function foo9: 3.0023088455200195ms


In [15]:
def cache(f):
    @functools.wraps(f)
    def deco(*args):
#         if args not in deco._cache:
#             result = f(*args)
#             deco._cache[args] = result
#         return deco._cache[args]
        if args in deco._cache:
            return deco._cache[args]
        result = f(*args)
        deco._cache[args] = result
        return result


    deco._cache = {}
    return deco

In [85]:
@profile()
@cache(max_limit=64)
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)

    
@profile()
@cache
def foo(n):
    time.sleep(n)

# foo(5)
# foo(6)

In [86]:
print(5, '->', fibo(1000))

Elapsed time for function fibo: 1.1205673217773438e-05ms
5 -> 5


In [5]:
from functools import lru_cache

@lru_cache(maxsize=64)
def foo():
    print('foo')

TypeError: lru_cache() got an unexpected keyword argument 'max_limit'

### Visibility scopes

In [2]:
# LEGB: Local, Enclosing, Global, Builtin

# Local
def foo():
    var = 42
    def foo_bar():
        print('foo_bar')
      
# print(var)
print(foo_bar())

NameError: name 'foo_bar' is not defined

In [5]:
# Global
import math

# print(log(-4.2))
# print(math.log(4.2))

var = 42

def foo():
#     print(var)
    global var
    var = 43
    print(var)
    
# def bar():
#     global var
#     print(var)
#     var += 1
#     print(var)

foo()
print(var)
# print(var)
# bar()

43
43


In [9]:
# Enclosing
# print('read')
# def foo():
#     var = 42
#     def bar():
#         print(var)
#     bar()
#     print(var)
# foo()

var = 41

print('write')
def foo():
    var = 42
    def bar():
        global var
        var = 43
        print(var)
#     print(var)
    bar()
    print(var)
foo()
print(var)

write
43
42
43


In [69]:
import builtins
print(dir(builtins))

print()

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

In [11]:
# Builtin
# LEGB
a = 4
b = 2
max = 42
print = 'print'
print(max)


print(max(1, -2, 0))

TypeError: 'str' object is not callable

In [2]:
# Name hiding (collisions)
# example with modules

# check out  a name from a list:
help('modules')


Please wait a moment while I gather a list of all available modules...



/home/dbhost/Work/python/python_course/env/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."


IPython             asynchat            ipykernel           rlcompleter
PIL                 asyncio             ipykernel_launcher  rmagic
__future__          asyncore            ipython_genutils    runpy
_abc                atexit              ipywidgets          sched
_ast                attr                itertools           secrets
_asyncio            audioop             itsdangerous        select
_bisect             autoreload          jedi                selectors
_blake2             backcall            jinja2              send2trash
_bootlocale         base64              json                setuptools
_bz2                bdb                 jsonschema          shelve
_codecs             binascii            jupyter             shlex
_codecs_cn          binhex              jupyter_client      shutil
_codecs_hk          bisect              jupyter_console     signal
_codecs_iso2022     bleach              jupyter_core        site
_codecs_jp          builtins            keyword   

/usr/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


### Recursion

In [3]:
# GNU - GNU is Not Unix
# Should have:
#   - Base or terminal case
#   - Self-Reference

# Left and right (tail) recursion

In [13]:
def fibo(n):
    """Inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)

print(fibo(1000))

RecursionError: maximum recursion depth exceeded

In [11]:
def rec_sum(n):
    if n==0:
        return 0
    result = rec_sum(n-1)
    result += n
    return result

print(rec_sum(10))

def rec_sum(n, acc=0):
    if n==0:
        return 0 + acc
    return rec_sum(n-1, n+acc)

print(rec_sum(10))

55
55


In [12]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [3]:
def reverse_l(s):
    head, tail = s[:1], s[1:]
    if tail:
        return reverse_l(tail) + head
    else:
        return head

print(reverse_l('abc'))

cba


In [4]:
def reverse_t(s, acc=''):
    head, tail = s[:1], s[1:]
    if tail:
        return reverse_t(tail, head + acc)
    else:
        return head + acc

print(reverse_t('abc'))

cba


In [37]:
def reverse_t2(s):
    head, tail, acc = s[:1], s[1:], ''
     
    while tail:
        head, tail = s[:1], s[1:]
        s, acc = tail, head + acc
    
    return acc

In [6]:
def fibo3(n):
    prev, curr, acc = 0, 1, 1
     
    while n > 2:
        prev, curr = curr, prev + curr
        n, acc = n-1, curr
    
    return acc

# 0 1 1 2 3 5 8
fibo3(1000)

26863810024485359386146727202142923967616609318986952340123175997617981700247881689338369654483356564191827856161443356312976673642210350324634850410377680367334151172899169723197082763985615764450078474174626

In [36]:
import string

print(reverse_t(string.ascii_lowercase))
print(reverse_l(string.ascii_lowercase))
print(reverse_t2(string.ascii_lowercase))

zyxwvutsrqponmlkjihgfedcba
zyxwvutsrqponmlkjihgfedcba
zyxwvutsrqponmlkjihgfedcba


In [1]:
import os

def traverse_dir(dir):
    total_size = 0
    for name in os.listdir(dir):
        path = os.path.join(dir, name)
        is_file = os.path.isfile(path)
        if os.path.isfile(path):
            total_size += os.path.getsize(path)
        else:
            total_size += traverse_dir(path)

    return total_size

In [2]:
dir_size = traverse_dir("/home/dbhost/Dropbox")
print(dir_size)

2012849279


### Magic attributes

In [16]:
class MyInt(int):
        
    def __add__(self, value):
        return super().__add__(int(value))
    
val = MyInt(42)
val + '1'


43

In [13]:
class context_manager():
    def __enter__(self):
        print('ENTER')
        return None

    def __exit__(self, type, value, traceback):
        print('EXIT')
        
with context_manager() as cm:
    print('hello')

ENTER
hello
EXIT


In [2]:
class coloured_print():
    
    def __init__(self):#, colour="31;40m"):
        self.old_print = None

    def __enter__(self):
        def my_print(*args):
            self.old_print('\x1B[31;40m', *args, '\x1B[0m')
        global print
        self.old_print = print
        print = my_print

    def __exit__(self, type, value, traceback):
        global print
        print = self.old_print
        
print('BEFORE')
with coloured_print() as aa:
    print('Hello')
    print('world', 42, [1, 2, 3])
print('AFTER')

BEFORE
 Hello 
 world 42 [1, 2, 3] 
AFTER


In [4]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

In [7]:
with timer('Elapsed: {}s'):
    time.sleep(1)

Elapsed: 1.001065969467163s


In [11]:
with open('test.txt', 'w+') as f:
    f.write('abc')
    
with open('test.txt', 'r') as f:
    print(f.read())

abc


In [ ]:
f = open('test.txt', 'r')
# 10/0
f.close()

### Links

1. Python Intorduciton: https://www.youtube.com/watch?v=5V7XG1mGiHc&list=PLlb7e2G7aSpTTNp7HBYzCBByaE1h54ruW